# Mutation Injection Notebook

Questo notebook analizza tutti i file JSON nella cartella `results` e, per ogni elemento in `results`, verifica se i campi di mutation testing (`mutation_score_percent`, `mutation_killed`, `mutation_survived`) sono presenti e non nulli.

Se mancano o sono nulli, viene eseguito il mutation testing tramite `mutmut_runner` e i risultati vengono aggiornati nel file JSON.

In [ ]:
import json
import os
import sys
from pathlib import Path

# Aggiungi la cartella src al path per importare i moduli
sys.path.insert(0, str(Path.cwd() / "src"))

from utils.mutmut_runner import get_mutation_metrics

In [ ]:
# Definizione delle cartelle
RESULTS_DIR = Path.cwd() / "results"
INPUT_CODE_DIR = Path.cwd() / "data" / "input_code"
OUTPUT_TESTS_DIR = Path.cwd() / "data" / "output_tests"

print(f"Results directory: {RESULTS_DIR}")
print(f"Input code directory: {INPUT_CODE_DIR}")
print(f"Output tests directory: {OUTPUT_TESTS_DIR}")

Results directory: /home/einrich99/Progetti/LLM-Agents-for-Collaborative-Test-Case/results
Input code directory: /home/einrich99/Progetti/LLM-Agents-for-Collaborative-Test-Case/data/input_code
Output tests directory: /home/einrich99/Progetti/LLM-Agents-for-Collaborative-Test-Case/data/output_tests


In [ ]:
def needs_mutation_testing(result: dict) -> bool:
    """
    Verifica se un result necessita di mutation testing.
    Ritorna True se i campi mutation_score_percent, mutation_killed, mutation_survived
    non esistono o sono null.
    """
    metrics = result.get("metrics", {})

    mutation_fields = ["mutation_score_percent", "mutation_killed", "mutation_survived"]

    for field in mutation_fields:
        if field not in metrics or metrics[field] is None:
            return True

    return False


def get_test_file_path(run_id: str, source_file: str) -> Path:
    """
    Costruisce il path del file di test corrispondente.
    """
    # Il nome del file di test è test_<nome_sorgente>.py
    test_file_name = f"test_{source_file}"
    test_file_path = OUTPUT_TESTS_DIR / run_id / test_file_name
    return test_file_path


def process_json_file(json_path: Path) -> bool:
    """
    Processa un singolo file JSON, eseguendo mutation testing dove necessario.
    Ritorna True se il file è stato modificato.
    """
    print(f"\n{'='*60}")
    print(f"Processing: {json_path.name}")
    print(f"{'='*60}")

    # Carica il JSON
    with open(json_path, "r") as f:
        data = json.load(f)

    run_id = data.get("run_id", "")
    results = data.get("results", [])

    modified = False

    for i, result in enumerate(results):
        source_file = result.get("file", "")
        status = result.get("status", "")

        print(f"\n[{i+1}/{len(results)}] {source_file}")

        # Skip se lo status non è success
        if status != "success":
            print(f"  ⏭️  Skipped (status: {status})")
            continue

        # Verifica se necessita mutation testing
        if not needs_mutation_testing(result):
            metrics = result.get("metrics", {})
            print(f"  ✅ Mutation data already present:")
            print(f"     Score: {metrics.get('mutation_score_percent')}%")
            print(f"     Killed: {metrics.get('mutation_killed')}")
            print(f"     Survived: {metrics.get('mutation_survived')}")
            continue

        # Costruisci i path
        source_path = INPUT_CODE_DIR / source_file
        test_path = get_test_file_path(run_id, source_file)

        # Verifica che i file esistano
        if not source_path.exists():
            print(f"  ❌ Source file not found: {source_path}")
            continue

        if not test_path.exists():
            print(f"  ❌ Test file not found: {test_path}")
            continue

        print(f"  🔬 Running mutation testing...")
        print(f"     Source: {source_path}")
        print(f"     Test: {test_path}")

        # Esegui mutation testing
        mutation_metrics = get_mutation_metrics(str(source_path), str(test_path))

        if mutation_metrics is None:
            print(f"  ⚠️  Mutation testing failed or timed out")
            # Imposta valori null nel caso di fallimento
            if "metrics" not in result:
                result["metrics"] = {}
            result["metrics"]["mutation_score_percent"] = None
            result["metrics"]["mutation_killed"] = None
            result["metrics"]["mutation_survived"] = None
        else:
            print(f"  ✅ Mutation testing completed:")
            print(f"     Score: {mutation_metrics['mutation_score_percent']}%")
            print(f"     Killed: {mutation_metrics['mutation_killed']}")
            print(f"     Survived: {mutation_metrics['mutation_survived']}")

            # Aggiorna le metriche
            if "metrics" not in result:
                result["metrics"] = {}
            result["metrics"].update(mutation_metrics)

        modified = True

    # Salva il JSON se modificato
    if modified:
        with open(json_path, "w") as f:
            json.dump(data, f, indent=4)
        print(f"\n💾 File saved: {json_path.name}")
    else:
        print(f"\n📄 No changes needed for: {json_path.name}")

    return modified

In [ ]:
# Trova tutti i file JSON nella cartella results
json_files = list(RESULTS_DIR.glob("*.json"))
print(f"Found {len(json_files)} JSON files in results folder:")
for f in json_files:
    print(f"  - {f.name}")


# Processa tutti i file JSON
modified_count = 0

for json_file in json_files:
    try:
        if process_json_file(json_file):
            modified_count += 1
    except Exception as e:
        print(f"\n❌ Error processing {json_file.name}: {e}")

print(f"\n{'='*60}")
print(f"SUMMARY")
print(f"{'='*60}")
print(f"Total files processed: {len(json_files)}")
print(f"Files modified: {modified_count}")

Found 1 JSON files in results folder:
  - single_llama8B_2026-01-16T17:15:07.json

Processing: single_llama8B_2026-01-16T17:15:07.json

[1/5] d10_hotel_reservation.py
  🔬 Running mutation testing...
     Source: /home/einrich99/Progetti/LLM-Agents-for-Collaborative-Test-Case/data/input_code/d10_hotel_reservation.py
     Test: /home/einrich99/Progetti/LLM-Agents-for-Collaborative-Test-Case/data/output_tests/single_llama8B_2026-01-16T17:15:07/test_d10_hotel_reservation.py
  ⚠️  Mutation testing failed or timed out

[2/5] d03_stack.py
  🔬 Running mutation testing...
     Source: /home/einrich99/Progetti/LLM-Agents-for-Collaborative-Test-Case/data/input_code/d03_stack.py
     Test: /home/einrich99/Progetti/LLM-Agents-for-Collaborative-Test-Case/data/output_tests/single_llama8B_2026-01-16T17:15:07/test_d03_stack.py
  ✅ Mutation testing completed:
     Score: 77.78%
     Killed: 14
     Survived: 4

[3/5] d04_linked_list.py
  🔬 Running mutation testing...
     Source: /home/einrich99/Progett